In [1]:
from sklearn import preprocessing
import numpy as np
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import norm

# Beat Histogram preprocessing

Change the `'Beat Histogram Overall Standard Deviation'` feature from a list to a single value by taking the average of values, Then normalize this value, using a max of 1.5

## Notes

The validity of this transformation needs to be verified.

In [336]:
def preProcessBeatHist(vals):
    avg = (sum(vals) / len(vals))
    return (avg)

In [337]:
def fileProcessor(F, trainingSize):
    x_train = []   #Training Data
    x_test = []    #Testing Data 
    y_train = []   #Training Classifiers
    y_test = []    #Testing Classifiers
    for file in F:    #F is a vector of file names
        x = [] #data
        y = [] #classifier
        tree = ET.parse(file) #Create XML file tree
        root = tree.getroot() #Get root of tree... In our case root is 'feature_vector_file'
        for child in root:    #Loop through subelements of root
            if child.tag == 'data_set':      #if subelement is 'data_set', perform:
                fileID = child.find('data_set_id').text   #'data_set_id' contains the classifer (i.e the composer name)
                composerName = fileID.split('/')[-2]      #extract composer name from 'data_set_id'
                y.append(composerName)                    #append composer name to classifier vector
                FeatureList = []                          #empty array for List of Features, Features have a name and set of values
                for feature in child.iter('feature'):  #Loop through 'data_set' subelement for features
                    v = []                                #create empty array to contain feature values
                    name = feature.find('name').text      #extract name of feature
                    vals = feature.findall('v')           #extract values of feature
                    for i in vals:                    #loop through extracted values
                        v.append(float(str(i.text).replace('E', 'e')))                  #append text of values to values array
                        #preprocessing.scale(v)             normalisation
#                         print(name)
                    if name == ('Beat Histogram Overall Average'):
#                         print(v)
                        v = preProcessBeatHist(v)
                    feat = [name, v]                  #join feature name and values
                    FeatureList.append(feat)          #append feature name and values to List of Features
                x.append(FeatureList)           #append List of Features for midi file being analysed to the array of data
        x_trainC, x_testC, y_trainC, y_testC = train_test_split(x, y,test_size=trainingSize) #split data into training and testing data
        #the following is done to ensure that each composer has an equal ratio of music split
        #into training and testing data... although the specific songs by each composer are chosen randomly
        #each composer has an equal ratio of their music split into testing and training data
        x_train = x_train + x_trainC    
        x_test = x_test + x_testC
        y_train = y_train + y_trainC
        y_test = y_test + y_testC
#     print(y_train)
    return [y_train, x_train]

In [258]:
v = ['beethovenValues.xml', 'chopinValues.xml']
[y,x] = fileProcessor(v, 0.33)
# preProcessBeatHist(x[0][2][1])

# Beat Histogram Normalization

In [259]:
def normPriorBeatHist(data):
    vals = []
    for i in range(len(data)):
        vals.append(data[i][6][1])
    # Fit a normal distribution to the data:
    [mu, std] = norm.fit(vals)
    return [mu, std]
#     plt.hist(vals, bins=25)
#     # Plot the PDF.
#     xmin, xmax = plt.xlim()
#     x = np.linspace(xmin, xmax, 100)
#     p = norm.pdf(x, mu, std)
#     plt.plot(x, p, 'k', linewidth=2)

In [417]:
def normBeatHistLikelihood(data, labels, composer):
    h = []
    for i in range(len(labels)):
        if labels[i] == composer:
            h.append(data[i][6][1])
    [mu, std] = norm.fit(h)
    return [mu, std]

In [433]:
v = ['beethovenValues.xml', 'chopinValues.xml']
[y,x] = fileProcessor(v, 0.33)
normBeatHistLikelihood(x,y,y[0])
[m, s] = normPriorBeatHist(x)

In [434]:
def calculateBeatLikelihood(value, model):
#     return norm(value[6][1], model[1]).cdf(model[0] + (0.125*model[1])) - norm(model[0], model[1]).cdf(value[6][1] - (0.125*model[1]))
    return norm(model[0], model[1]).pdf(value[6][1])

In [435]:
calculateBeatLikelihood(x[0], [m,s])

1.0874166105637393